rozpoznawanie typów reklamacji

In [ ]:
import sys, os, re, time
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

# import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics

from imblearn.over_sampling import RandomOverSampler

import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

from tensorflow.keras import backend as K

In [ ]:
K.set_session(tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=4, inter_op_parallelism_threads=4)))

Ładowanie danych z pliku xlsx

In [ ]:
datapath='/mnt/c/dev/reklamacje/'
datafile='reklamacje_20181106_train.xlsx'
dane_surowe=pd.read_excel(os.path.join(datapath,datafile))


In [ ]:
#dane_surowe.info()

In [ ]:
dane_surowe.columns

In [ ]:
input_data=pd.DataFrame()
input_data[['content','category']]=dane_surowe[['tresc_zgl','typ_train']]

In [ ]:
input_data.describe()

In [ ]:
# clearing data
# remove duplicates
input_data.drop_duplicates(inplace=True)
# remove empty
input_data=input_data.dropna()

In [ ]:
# find duplicates
input_data[input_data.duplicated(keep=False)]

In [ ]:
# input_data['content']

załadowanie słowników tłumaczeń

In [ ]:
# Słownik synonimów / podmian

podmiany=pd.read_excel(os.path.join(datapath,'roboczy_slownik_synonimow.xlsx'))

preprocessing of content text

In [ ]:
def preprocess_texts(raw_texts,replacements):
    """
    texts: np.Series containing strings to be preprocessed
    replacements: pairs of what convert to what
    return np.Series with corrected texts
    """
    resulttext=raw_texts.str.lower()
    for [co,naco] in replacements.values:
       resulttext=resulttext.str.replace(re.compile(str(co)),str(naco))
    return resulttext


In [ ]:
prep_texts=preprocess_texts(input_data['content'],podmiany)

In [ ]:
prep_texts.head()

In [ ]:
#type(prep_texts)

In [ ]:
# prep_texts.to_csv(os.path.join(datapath,'texts_for_emb.txt'))
# incorrect <- validation set użyty do nauki - trzeba poprawić
# prep_texts.to_csv(os.path.join(datapath,'texts_for_emb.txt'),sep='\n',index=False)

In [ ]:
def preprocess_labels(raw_labels,interesting_labels):
    """
    raw_labels: np.Series with labels
    interesting_labels: list of labels you are interested in
    
    returns np.Series with corrected labels
    """
    other_label='OTHER'
    result_labels=raw_labels
    result_labels=result_labels.apply(lambda x: x if x in interesting_labels else other_label)
    return result_labels

In [ ]:
interesting_categories=['XDR','XOA','XRF','XSP']
prep_labels=preprocess_labels(input_data['category'],interesting_categories)

In [ ]:
type(prep_labels)

In [ ]:
def categorize_labels(labels,dictionary=None):
    if dictionary==None:
        cat_labels, uniques = pd.factorize(labels)
    else:
        None # dorobić mapowania jeśli słownik już był podany
    return cat_labels, uniques

In [ ]:
cat_labels,label_dict=categorize_labels(prep_labels)


In [ ]:
# pd.Series(cat_labels).value_counts()

In [ ]:
#texts_train.shape


In [ ]:
# save dictionary to the disk
pd.DataFrame(label_dict).to_excel(os.path.join(datapath,'slownik_kategorii.xlsx'))

split data to train and validation parts

In [ ]:
texts_train,texts_val,y_train,y_val=train_test_split(prep_texts.values,
                                                     cat_labels,test_size=0.25,random_state=1, shuffle=True)

In [ ]:
#save file for fasttext
pd.Series(texts_train).to_csv(os.path.join(datapath,'texts_for_emb.txt'),sep='\n',index=False)

In [ ]:
# oversampling to boost minority classes
ros = RandomOverSampler(random_state=0)

texts_train, y_train = ros.fit_resample(np.reshape(texts_train, (-1, 1)),y_train)

# shuffle to be sure 
texts_train, y_train = shuffle(texts_train, y_train, random_state=0)

texts_train=texts_train.reshape(-1)

In [ ]:
# prepare files for fasstext
fasttext_train_set=[]
for i,t in enumerate(y_train):
    fasttext_train_set.append('__label__'+str(t)+' '+texts_train[i])

fasttext_val_set=texts_val
    

In [ ]:
pd.Series(fasttext_train_set).to_csv(os.path.join(datapath,'fasttext_train_set.txt'),sep='\n',index=False)
pd.Series(fasttext_val_set).to_csv(os.path.join(datapath,'fasttext_val_set.txt'),sep='\n',index=False)

In [ ]:
# teraz odpalamy fasttext i w wyniku otrzymujemy predictions.txt

In [ ]:
y_val_predicted=pd.read_csv(os.path.join(datapath,'predictions.txt'), header=None)

In [ ]:
drop_fasttext_label = lambda x: int(re.sub('__label__','',x))

In [ ]:
y_val_predicted=y_val_predicted.applymap(drop_fasttext_label)

In [ ]:
y_val_pred=y_val_predicted.values
#print(y_val_predicted)
print(y_val_pred.shape)

In [ ]:
# oversampling also validation set 
#texts_val, y_val = ros.fit_resample(np.reshape(texts_val, (-1, 1)),y_val)
#texts_val=texts_val.reshape(-1)

In [ ]:

print(texts_train.shape)
print(y_train.shape)
print(texts_val.shape)
print(y_val.shape)

Data analysis

In [ ]:
# oversampling to boost minority classes
# ros = RandomOverSampler(random_state=0)

# x_train, y_train = ros.fit_resample(x_train,y_train)


create mlp model

calculate statistics per class

In [ ]:
#y_prob = mymodel.predict(x_val) 
#y_preds = y_prob.argmax(axis=-1)
y_preds = y_val_pred
cm = metrics.confusion_matrix(y_val, y_preds)

In [ ]:
print(metrics.classification_report(list(y_val),list(y_preds),labels=[1,2,3,4],target_names=label_dict[1:]))
#print(metrics.classification_report(list(y_val),list(y_preds),target_names=label_dict))

In [ ]:
cm

In [ ]:
df_cm = pd.DataFrame(cm, label_dict, label_dict)
plt.figure(figsize = (10,7))
sn.set(font_scale=1.0)#for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 12})# font size

In [ ]:
pd.Series(y_val).value_counts()

In [ ]:
print(mymodel.summary())

In [ ]:
y_prob_train = mymodel.predict(x_train) 
y_preds_train = y_prob_train.argmax(axis=-1)
cm_train = metrics.confusion_matrix(y_train, y_preds_train)

In [ ]:
cm_train

In [ ]:
df_cm_train = pd.DataFrame(cm_train, label_dict, label_dict)
plt.figure(figsize = (10,7))
sn.set(font_scale=1.0)#for label size
sn.heatmap(df_cm_train, annot=True, annot_kws={"size": 12})# font size

In [ ]:
print(metrics.classification_report(list(y_train),list(y_preds_train),labels=[1,2,3,4],target_names=label_dict[1:]))

In [ ]:
def tune_ngram_model(data):
    """Tunes n-gram model on the given dataset.

    # Arguments
        data: tuples of training and test texts and labels.
    """
    
    # Select parameter values to try.
    num_layers = [1, 2, 3]
    num_units = [4, 6, 10]
    #dropouts =[0.3,0.4,0.5]

    # Save parameter combination and results.
    params = {
        'layers': [],
        'units': [],
        'accuracy': [],
        'loss':[],
        'f1':[],
    }
    
    (x_train, y_train), (x_val, y_val) = data

    # Iterate over all parameter combinations.
    for layers in num_layers:
        for units in num_units:
                params['layers'].append(layers)
                params['units'].append(units)
                print(f'parameters: layers-{layers}, units-{units}')
                myaccuracy, myloss, mymodel = train_ngram_model(data,
                      num_classes=len(label_dict),
                      learning_rate=4e-3,
                      epochs=7,
                      batch_size=128,
                      layers=layers,
                      units=units,
                      dropout_rate=0.4,
                      l2=0.005)
                y_prob = mymodel.predict(x_val) 
                y_preds = y_prob.argmax(axis=-1)
                myf1=metrics.f1_score(list(y_val),list(y_preds),labels=[1,2,3,4])
                print((f'Accuracy: {myaccuracy}, Loss: {myloss}, F1: {myf1}, Parameters: (layers={layers}, units={units})'))
                params['accuracy'].append(myaccuracy)
                params['loss'].append(myloss)
                params['f1'].append(myf1)
    #_plot_parameters(params)
    return params
    
def _plot_parameters(params):
    """Creates a 3D surface plot of given parameters.

    # Arguments
        params: dict, contains layers, units and accuracy value combinations.
    """
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_trisurf(params['layers'],
                    params['units'],
                    params['accuracy'],
                    cmap=cm.coolwarm,
                    antialiased=False)
    plt.show()

In [ ]:
wyniki = tune_ngram_model(mydata)